# Objective:
    1. Show how the dataset is structured
    2. Explore a bit about customer behaviour based on the data and do basic customer segmentation
    3. Recommendations for future analysis

This dataset was taken from the Retail Rocket Recommender System dataset: https://www.kaggle.com/retailrocket/ecommerce-dataset/home

And data was between June 2, 2015 and August 1, 2015

In [160]:
import pandas as pd
import numpy as np

import datetime 
import time

%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from tqdm.auto import tqdm

Let us load the Retail Rocket CSV files into DataFrames

In [20]:
events_df = pd.read_csv('data/events.csv')
category_tree_df = pd.read_csv('data/category_tree.csv')
item_properties_1_df = pd.read_csv('data/item_properties_part1.csv')
item_properties_2_df = pd.read_csv('data/item_properties_part2.csv')

# Let's take a peek at the Events dataframe

In [21]:
events_df.head()

timestamp  visitorid event  itemid  transactionid
0  1433221332117     257597  view  355908            NaN
1  1433224214164     992329  view  248676            NaN
2  1433221999827     111016  view  318965            NaN
3  1433221955914     483717  view  253185            NaN
4  1433221337106     951259  view  367447            NaN

The timestamp portion is in Unix Epoch format e.g. 1433221332117 will be converted to Tuesday, 2 June 2015 5:02:12.117 AM GMT

Visitor Id is the unique user currently browsing the website

Event is what the user is currently doing in that current timestamp

Transaction ID will only have value if the user made a purchase as shown below

In [22]:
#Which event has a value in its transaction id
events_df[events_df.transactionid.notnull()].event.unique()

array(['transaction'], dtype=object)

The rest of the events with NaN transaction ids are either view or add to cart

In [23]:
#Which event/s has a null value
events_df[events_df.transactionid.isnull()].event.unique()

array(['view', 'addtocart'], dtype=object)

# Now let's take a look at the Item Properties

In [24]:
item_properties_1_df.head()

timestamp  itemid    property                            value
0  1435460400000  460429  categoryid                             1338
1  1441508400000  206783         888          1116713 960601 n277.200
2  1439089200000  395014         400  n552.000 639502 n720.000 424566
3  1431226800000   59481         790                       n15360.000
4  1431831600000  156781         917                           828513

Timestamp is still the same Unix / Epoch format

Item id will be the unique item identifier

Property is the Item's attributes such as category id and availability while the rest are hashed for confidentiality purposes

Value is the item's property value e.g. availability is 1 if there is stock and 0 otherwise

Note: Values that start with "n" indicate that the value preceeding it is a number e.g. n277.200 is equal to 277.2

# Category IDs

In [25]:
category_tree_df.head()

categoryid  parentid
0        1016     213.0
1         809     169.0
2         570       9.0
3        1691     885.0
4         536    1691.0

Category IDs explain the relationship of different products with each other e.g. Category ID 1016 is a child of Parent ID 213.

Below shows the number of items under category id 1016

In [26]:
item_properties_1_df.loc[(item_properties_1_df.property == 'categoryid') & (item_properties_1_df.value == '1016')].sort_values('timestamp').head()

timestamp  itemid    property value
2105129  1431226800000  377283  categoryid  1016
970935   1431226800000  362999  categoryid  1016
3826363  1431226800000  373242  categoryid  1016
6363096  1431226800000  339403  categoryid  1016
4497603  1431226800000  390209  categoryid  1016

# Customer behaviour exploration

I think it's prudent to start separating customers into two categories, those who purchased something and those who didn't

In [27]:
#Let's get all the customers who bought something
customer_purchased = events_df[events_df.transactionid.notnull()].visitorid.unique()
customer_purchased.size

11719

Assumptions:
    1. Since we have no information whether there were any repeat users who bought something from the site, I'll just have to assume for now that the 11,719 visitors are unique and made at least a single purchase

In [28]:
#Let's get all unique visitor ids as well
all_customers = events_df.visitorid.unique()
all_customers.size

1407580

Out of 1,407,580 unique visitor ids, let's take out the ones that bought something

In [29]:
customer_browsed = [x for x in all_customers if x not in customer_purchased]

In [30]:
len(customer_browsed)

1395861

So there were actually 1,395,861 unique site visitors who didn't buy anything, again assuming that there were no repeat users with different visitor IDs

In [31]:
#Another way to do it using Numpy
temp_array = np.isin(customer_browsed, customer_purchased)
temp_array[temp_array == False].size

1395861

In [32]:
#A sample list of the customers who bought something
customer_purchased[:10]

array([ 599528,  121688,  552148,  102019,  189384,  350566,  404403,
        505565,  945184, 1406787])

# Below is a snapshot of visitor id 102019 and their buying journey from viewing to transaction (purchase)

In [33]:
events_df[events_df.visitorid == 102019].sort_values('timestamp')

timestamp  visitorid        event  itemid  transactionid
19690  1433175714335     102019         view   49521            NaN
19501  1433175801314     102019    addtocart   49521            NaN
14842  1433175812596     102019         view  150318            NaN
19573  1433175871497     102019         view   49521            NaN
8701   1433175894837     102019         view   49521            NaN
19708  1433175945872     102019         view  150318            NaN
8740   1433176042269     102019         view   49521            NaN
814    1433176736375     102019  transaction  150318        13556.0
19724  1433176736422     102019  transaction   49521        13556.0

If we want to convert the UNIX / Epoch time format to readable format then just do the code below

In [34]:
tz = int('1433221332')
new_time = datetime.datetime.fromtimestamp(tz)
new_time.strftime('%Y-%m-%d %H:%M:%S')

'2015-06-02 08:32:12'

In [35]:
tz = int('1438400163')
new_time = datetime.datetime.fromtimestamp(tz)
new_time.strftime('%Y-%m-%d %H:%M:%S')

'2015-08-01 07:06:03'

# What insights can we offer the visitor to guide them in their buying journey?

-perhaps we can offer them a list of what previous visitors bought together with the item they are currently viewing

In [36]:
# Firstly let's create an array that lists visitors who made a purchase
customer_purchased = events_df[events_df.transactionid.notnull()].visitorid.unique()
    
purchased_items = []
    
# Create another list that contains all their purchases 
for customer in customer_purchased:

    #Generate a Pandas series type object containing all the visitor's purchases and put them in the list
    purchased_items.append(list(events_df.loc[(events_df.visitorid == customer) & (events_df.transactionid.notnull())].itemid.values))                                  

So now all items purchased together are presented as a list of lists, shown below are the first 5 samples

In [37]:
purchased_items[:5]

[[np.int64(356475)],
 [np.int64(15335),
  np.int64(380775),
  np.int64(237753),
  np.int64(317178),
  np.int64(12836),
  np.int64(400969),
  np.int64(105792),
  np.int64(25353),
  np.int64(200793),
  np.int64(80582),
  np.int64(302422)],
 [np.int64(81345)],
 [np.int64(150318), np.int64(49521)],
 [np.int64(310791), np.int64(299044)]]

In [38]:
# Write a function that would show items that were bought together (same of different dates) by the same customer
def recommender_bought_bought(item_id, purchased_items):
    
    # Perhaps implement a binary search for that item id in the list of arrays
    # Then put the arrays containing that item id in a new list
    # Then merge all items in that list and get rid of duplicates
    recommender_list = []
    for x in purchased_items:
        if item_id in x:
            recommender_list += x
    
    #Then merge recommender list and remove the item id
    recommender_list = list(set(recommender_list) - set([item_id]))
    
    return recommender_list

# So now we can present to the visitor a list of the other items a customer previously bought along with what item the current visitor is viewing e.g. item number 302422

In [39]:
recommender_bought_bought(302422, purchased_items)

[np.int64(105792),
 np.int64(200793),
 np.int64(12836),
 np.int64(80582),
 np.int64(380775),
 np.int64(15335),
 np.int64(400969),
 np.int64(25353),
 np.int64(237753),
 np.int64(317178)]

That was a very crude way of recommending other items to the visitor

# What other insights can we gather from the items that were viewed, added to cart and sold?

# Can we perhaps cluster the visitors and see if classes appear?

For that I will need to create a new dataframe and engineer a few features for it

How many unique visitors did we have for the site from June 2, 2015 to August 1, 2015?

Shown below are the total number of visitors for that time duration (was also shown at the close to the start of this paper)

In [40]:
#Put all the visitor id in an array and sort it ascendingly
all_visitors = events_df.visitorid.sort_values().unique()
all_visitors.size

1407580

In [41]:
buying_visitors = events_df[events_df.event == 'transaction'].visitorid.sort_values().unique()
buying_visitors.size

11719

Out of 1,407,580 visitors, ony 11,719 bought something so around 1,395,861 visitors just viewed items

In [42]:
viewing_visitors_list = list(set(all_visitors) - set(buying_visitors))


Now lets create a function that creates a dataframe with new features: visitorid, number of items viewed, total viewcount, bought something or not

In [43]:
def create_dataframe(visitor_list):
    
    array_for_df = []
    for index in visitor_list:

        #Create that visitor's dataframe once
        v_df = events_df[events_df.visitorid == index]

        temp = []
        #Add the visitor id
        temp.append(index)

        #Add the total number of unique products viewed
        temp.append(v_df[v_df.event == 'view'].itemid.unique().size)

        #Add the total number of views regardless of product type
        temp.append(v_df[v_df.event == 'view'].event.count())

        #Add the total number of purchases
        number_of_items_bought = v_df[v_df.event == 'transaction'].event.count()
        temp.append(number_of_items_bought)

        #Then put either a zero or one if they made a purchase
        if(number_of_items_bought == 0):
            temp.append(0)
        else:
            temp.append(1)

        array_for_df.append(temp)
    
    return pd.DataFrame(array_for_df, columns=['visitorid', 'num_items_viewed', 'view_count', 'bought_count', 'purchased'])

Let's apply this to buying visitors first

In [44]:
buying_visitors_df = create_dataframe(buying_visitors)

In [59]:
buying_visitors_df.head()

visitorid  num_items_viewed  view_count  bought_count  purchased
0        172                22          33             2          1
1        186                 1           2             1          1
2        264                 2           3             2          1
3        419                 3           4             1          1
4        539                 1           4             1          1

In [45]:
buying_visitors_df.shape

(11719, 5)

I think I'll only get around 27,821 samples from the viewing visitors list so that there is a 70-30 split for training and test data. 

In [46]:
#Let's shuffle the viewing visitors list for randomness
import random
random.shuffle(viewing_visitors_list)

In [47]:
viewing_visitors_df = create_dataframe(viewing_visitors_list[0:27820])

In [60]:
viewing_visitors_df.head()

visitorid  num_items_viewed  view_count  bought_count  purchased
0     164699                 1           3             0          0
1     519822                 2           5             0          0
2    1130749                 2           3             0          0
3    1243653                 1           2             0          0
4     912688                 1           1             0          0

In [48]:
viewing_visitors_df.shape

(27820, 5)

Now let's combine the two dataframes

In [49]:
main_df = pd.concat([buying_visitors_df, viewing_visitors_df], ignore_index=True)

In [61]:
main_df.head()

visitorid  num_items_viewed  view_count  bought_count  purchased
33319     602598                 2           2             0          0
32716      79068                 3           3             0          0
11396    1366782                 1           2             1          1
5161      632358                 8          13             1          1
7111      865129                 2           3             1          1

Let's plot main_df and see if anything comes up

In [50]:
#Let's shuffle main_df first
main_df = main_df.sample(frac=1)

In [51]:
#Plot the data
sns.pairplot(main_df, x_vars = ['num_items_viewed', 'view_count', 'bought_count'],
             y_vars = ['num_items_viewed', 'view_count', 'bought_count'],  hue = 'purchased')

The plot above clearly indicates that the higher the view count, the higher the chances of that visitor buying something. Duh!

# Since the relationship is Linear, let's try a simple Logistic Regression model to predict future visitor purchase behaviour

We separate the features (drop visitorid since it's categorical data and bought count) and the target (which is whether the visitor bought something or not)

In [ ]:
X = main_df.drop(['purchased', 'visitorid', 'bought_count'], axis = 'columns')
y = main_df.purchased

In [215]:
from imblearn.over_sampling import SMOTE
from collections import Counter

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size= 0.2)

sm = SMOTE(random_state=42)
X_train, y_train = sm.fit_resample(X_train, y_train)


In [216]:
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier

# ratio = (y_train.value_counts()[0] / y_train.value_counts()[1])

xgbModel = XGBClassifier(
    n_estimators=700,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    eval_metric="mlogloss",
    use_label_encoder=False,
    # scale_pos_weight=ratio
)


rf = RandomForestClassifier(
    n_estimators=700,
    max_depth=5,
    random_state=42,
)

logreg = LogisticRegression(max_iter=1000, random_state=42)
knn = KNeighborsClassifier(n_neighbors=7)

estimators = [("xgb", xgbModel), ("rf", rf), ("logreg", logreg), ("knn", knn)]

stack_model = StackingClassifier(
    estimators=estimators,
    final_estimator=XGBClassifier(
        n_estimators=700,
        max_depth=5,
        random_state=42,
        learning_rate=0.05,
    ),
    passthrough=True,
    n_jobs=-1,
)

In [217]:
stack_model.fit(X_train, y_train)

StackingClassifier(estimators=[('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=0.8, device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric='mlogloss',
                                              feature_types=None,
                                              feature_weights=None, gamma=None,
                                              grow_policy=None,
                                              importance_type=None,
                                              interaction_...
                                                 gamma=None, grow_policy=None,
                                                 importance_type=None,
                                                 interaction_constraints=None,
                                                 learning_rate=0.05,
                                                 max_bin=None,
                                                 max_cat_threshold=None,
                                                 max_cat_to_onehot=None,
                                                 max_delta_step=None,
                                                 max_depth=5, max_leaves=None,
                                                 min_child_weight=None,
                                                 missing=nan,
                                                 monotone_constraints=None,
                                                 multi_strategy=None,
                                                 n_estimators=700, n_jobs=None,
                                                 num_parallel_tree=None, ...),
                   n_jobs=-1, passthrough=True)

In [218]:
y_train_stack_model = stack_model.predict(X_train)
y_test_stack_model = stack_model.predict(X_test)

In [ ]:
print("train data : ")
print("Accuracy:", metrics.accuracy_score(y_train, y_train_stack_model))
print("\nClassification Report:\n", metrics.classification_report(y_train, y_train_stack_model))
print("\nConfusion Matrix:\n", metrics.confusion_matrix(y_train, y_train_stack_model))
print("\ntest data : ")
print("Accuracy:", metrics.accuracy_score(y_test, y_test_stack_model))
print("\nClassification Report:\n", metrics.classification_report(y_test, y_test_stack_model))
print("\nConfusion Matrix:\n", metrics.confusion_matrix(y_test, y_test_stack_model))

train data : 
Accuracy: 0.7846198777857656

Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.82      0.79     22256
           1       0.81      0.74      0.78     22256

    accuracy                           0.78     44512
   macro avg       0.79      0.78      0.78     44512
weighted avg       0.79      0.78      0.78     44512


Confusion Matrix:
 [[18350  3906]
 [ 5681 16575]]

test data : 
Accuracy: 0.8028578654527061

Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.83      0.86      5564
           1       0.65      0.74      0.69      2344

    accuracy                           0.80      7908
   macro avg       0.76      0.78      0.77      7908
weighted avg       0.81      0.80      0.81      7908


Confusion Matrix:
 [[4618  946]
 [ 613 1731]]


<!-- # So our model's accuracy in predicting buying visitors is around 79.46% -->

The graph above shows the accuracy of our binary classifier (Logistic Regression). Just means that the closer the orange curve leans to the top left hand part of the graph, the better the accuracy.